In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
plt.style.use('seaborn-whitegrid')

import seaborn as sns

from collections import Counter

import warnings
warnings.filterwarnings('ignore')

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:

suicide_rates=pd.read_csv('/kaggle/input/suicide-rates-overview-1985-to-2016/master.csv')

In [ ]:
suicide_rates.head(20)

In [ ]:
suicide_rates.columns

In [ ]:
suicide_rates.describe()

<a id='2'></a>
## Variable Description

 <font color='red'>  
* country:place of suicide
* year:when the suicide happened
* sex:gender of the person who committed suicide
* age:age
* suicides_no
* population: the population of the country where she/he lives  
* suicides/100k pop: The ratio of suicide rate to 100 thousand of the population.
* country-year:we will don't use
* HDI for year:Human Development Index-we will don't use
* gdp_for_year:Gross domestic product(is a monetary measure of the market value of all the final goods and services produced in a specific time period) for year
* gdp_per_capita:Gross Domestic Product per capita shows a country's GDP divided by its total population.(Gross domestic product/total population)
* generation:
    * G.I Generation(Greatest Generation):1901-1927
    * Silient:1928-1945
    * Boomers:1946-1964
    * Generation X:1965-1980
    * Millenials(Generation Y):1981-1996
    * Generation Z:1997-?*
    

In [ ]:
# Let's take out what we will not use.
del suicide_rates['country-year']
del suicide_rates['HDI for year']


In [ ]:
suicide_rates.info()

* float64(1):suicides/100k pop
* int64(4) :year,suicides_no,population,gdp_per_capita
* object(5):country,sex,age,gdp_for_year,generation

<a id='4'></a>
## Outlier Detection

In [ ]:
def detect_outliers(df,columnNames):
    #df=dataframe for example suicide_rates
    outlier_indices=[]
    for c in columnNames:
        #1st Quartile
        Q1=np.percentile(df[c],25)
        #3rd Quartile
        Q3=np.percentile(df[c],75)
        #IQR
        IQR=Q3-Q1
        #Outlier Step
        outlier_step=IQR * 1.5
        #detect outlier and ther indices
        outlier_list_col=df[(df[c]<Q1-outlier_step) | (df[c]>Q3+outlier_step)].index
        #store indeces
        outlier_indices.extend(outlier_list_col)
        
    outlier_indices=Counter(outlier_indices)
    multiple_outliers=list(i for i,v in outlier_indices.items() if v>1)
    return multiple_outliers

In [ ]:
suicide_rates.loc[detect_outliers(suicide_rates,['population','gdp_per_capita ($)','suicides/100k pop'])]

In [ ]:
#drop outliers
suicide_rates=suicide_rates.drop(detect_outliers(suicide_rates,['population','gdp_per_capita ($)','suicides/100k pop']),axis=0).reset_index(drop=True)

<font color='red'>
    
<a id='6'></a>
# What I Wonder

<font color='red'>
    
<a id='6.1'></a>
### In which year did the most suicides occur

In [ ]:
year_list=list(suicide_rates.year.unique())
year_suicide_ratio=[]
for i in year_list:
    a=suicide_rates[suicide_rates['year']==i]
    year_suicide_rate=sum(a['suicides/100k pop'])/len(a)
    year_suicide_ratio.append(year_suicide_rate)
data=pd.DataFrame({'year_list':year_list,'year_suicide_ratio':year_suicide_ratio})
new_index=(data['year_suicide_ratio'].sort_values(ascending=True)).index.values
sorted_data=data.reindex(new_index)


# visualization
plt.figure(figsize=(15,10))
sns.barplot(x=sorted_data['year_list'],y=sorted_data['year_suicide_ratio'])
plt.xticks(rotation=90)
plt.xlabel('Years')
plt.ylabel('Suicides')
plt.title('Which year did the most suicides occour')
plt.show()

<font color='red'>  
Hmmm..
suicide at most in 1995

[A few things that happened in 1995](https://en.wikipedia.org/wiki/1995#Events)


    
<a id='6.2'></a>
### Which country has the highest suicide rate for the 100,000 population

In [ ]:
country_list=list(suicide_rates.country.unique())
country_suicide_ratio=[]

for i in country_list:
    a=suicide_rates[suicide_rates['country']==i]
    country_suicide_rate=sum(a['suicides/100k pop'])/len(a)
    country_suicide_ratio.append(country_suicide_rate)

data=pd.DataFrame({'country_list':country_list,'country_suicide_ratio':country_suicide_ratio})
new_index=(data['country_suicide_ratio'].sort_values(ascending=False)).index.values
sorted_data=data.reindex(new_index)


# visualization
plt.figure(figsize=(15,10))
sns.barplot(x=sorted_data['country_list'],y=sorted_data['country_suicide_ratio'])
plt.xticks(rotation=90)
plt.xlabel('Countrys')
plt.ylabel('Suicide Rate')
plt.title('Which country has the highest suicide rate for the 100,000 population')
plt.show()
     
    

 <font color='red'>  
Why Lithuania's suicide rate is much higher than in other countries???

For a number of reasons:
* the sun does not shine.
* the weather is rainy.
* high alcohol consumption.


Why Sri Lanka's suicide rate is much higher than in other countries???
* Unfortunately, purely for economic reasons...:(

Why Hungary's suicide rate is much higher than in other countries???

* Radio Free Europe, an American station broadcasting to Hungary, puts the blame for the high suicide rate on the failings of Hungary's Communist system.


    
<a id='6.3'></a>
### How old were those who committed suicide

In [ ]:
suicide_rates[['age','suicides/100k pop']].groupby(['age'],as_index=False).mean().sort_values(by='suicides/100k pop',ascending=False)

<font color='red'>  
Why do old people commit suicide?

* Older adult suicide is often triggered by elders’ loss of control over health conditions or financial circumstances that results in feelings of hopelessness.

Why do young people commit suicide?
* Academic failure
* Being arrested or imprisoned
* Bullying, shaming, or humiliation, including cyberbullying
* Financial problems
* End of a close friendship or romantic relationship
* Job loss
* Loss of friends or family acceptance due to revealing your sexual orientation
* Loss of social status


<a id='6.4'></a>
### Gender ratio of those who committed suicide

In [ ]:
suicide_rates[['sex','suicides/100k pop']].groupby(['sex'],as_index=False).mean().sort_values(by='suicides/100k pop',ascending=False)

In [ ]:
plt.figure(figsize=(8,4))
sns.barplot(x=suicide_rates['sex'],y=suicide_rates['suicides/100k pop'])
plt.xticks(rotation=90)
plt.xlabel("Sex")
plt.ylabel("Suicide Rate")
plt.title("Gender ratio of those who committed suicide")
plt.show()

Summary:
Women also are even more likely than men to attempt suicide but male suicide methods are often more violent, making them more likely to be completed before anyone can intervene. Access to means is a big contributing factor: in the US for example, six-in-10 gun owners are men – and firearms account for more than half of suicides.


    
<a id='6.5'></a>
### Suicide rate of generations

In [ ]:
suicide_rates[['generation','suicides/100k pop']].groupby(['generation'],as_index=False).mean().sort_values(by='suicides/100k pop',ascending=False)

In [ ]:
plt.figure(figsize=(15,10))
sns.barplot(x=suicide_rates['generation'],y=suicide_rates['suicides/100k pop'])
plt.xticks(rotation=90)
plt.xlabel("Generations")
plt.ylabel("Suicides Rates")
plt.title("Suicide rate of generations")
plt.show()

<font color='red'>  
You don't need to go up, I'm here for you :D

* G.I Generation(Greatest Generation):1901-1927
* Silient:1928-1945
* Boomers:1946-1964
* Generation X:1965-1980
* Millenials(Generation Y):1981-1996
* Generation Z:1997-?*

In [ ]:
suicide_rates.head()

In [ ]:
suicide_rates.columns

In [ ]:
year_list=list(suicide_rates['gdp_per_capita ($)'].unique())
year_suicide_ratio=[]
for i in year_list:
    a=suicide_rates[suicide_rates['gdp_per_capita ($)']==i]
    year_suicide_rate=sum(a['suicides/100k pop'])/len(a)
    year_suicide_ratio.append(year_suicide_rate)
data=pd.DataFrame({'year_list':year_list,'year_suicide_ratio':year_suicide_ratio})
new_index=(data['year_suicide_ratio'].sort_values(ascending=True)).index.values
sorted_data=data.reindex(new_index)


# visualization
plt.figure(figsize=(15,10))
sns.barplot(x=sorted_data['year_list'],y=sorted_data['year_suicide_ratio'])
plt.xticks(rotation=90)
plt.xlabel('Years')
plt.ylabel('gdp_per_capita ($)')
plt.title('Which year did the most suicides occour')
plt.show()